# Lab 1: Modelado de datos en Elasticsearch

## Introducción

En esta práctica vamos a aprender a modelar los datos en Elasticsearch explorando las diferentes alternativas para crear un mapping type usando la API de Elasticsearch y la interfaz gráfica de Kibana:

* Obtener información del cluster de Elasticsearch.
* Explorar la información asociada a un índice.
* Insertar datos en Elasticserach.
* Definir un mapping type para índice.
* Usar dynamic templates para definir el mapping type de un índice.
* CRUD en Elasticsearch.

En la maquina virtual que estás utilizando para realizar estas prácticas hay instalado un nodo de Elasticsearch y la versión correspondiente de Kibana.

* Para acceder al cluster de Elasticserach puedes utilizar la dirección http://127.0.0.1:9200
* Para acceder a Kibana pudes utilizar la dirección http://127.0.0.1:5601

## Información del cluster

Antes de empezar con el modelado de datos en Elasticsearch, vamos a ver como extraer información del cluster de elasticsearch.

Si accedemos driectamente a la URL raíz del cluster obtenemos información sobre la instalación del cluster, como el nombre del cluster, la versión de Elasticsearch instalda o la versión de Lucene sobre la que está construida la versión de Elasticsearch.

Vamos a ver esta infromación ejecutando un curl desde consola:

In [ ]:
!curl -X GET http://elasticsearch:9200

Con los datos obtenidos intenta contestar a estas preguntas:

1. ¿Qué nombre tiene el cluster?
2. ¿Qué versión de Elasticsearch hay instalada en la máquina virutal?
3. ¿Sobre qué versión de Lucene está desarrollada esta versión de Elasticsearch?
4. ¿Con qué versión de Elasticsearch son retrocompatibles los índices ya creados? 

Si lo que queremos es comprobar cuál es el estado de salud del cluster realizaremos la siguiente petición a la API REST de Elasticsearch:

In [ ]:
!curl -X GET http://elasticsearch:9200/_cluster/health?pretty

Si queremos obtener información sobre los índices creados en Elasticsearch pediremos la siguiente URL al cluster:

In [ ]:
!curl -X GET http://elasticsearch:9200/_cat/indices?expand_wildcards=all

Las columnas mostradas hacen referencia a la siguiente información:
* health: Salud del índice (green, yellow o red).
* status: Estado del índice (open, close).
* index: Nombre del índice.
* uuid: Identificador único del índice.                 
* pri: Número del shards primarios.
* rep: Número de réplicas del índice.
* docs.count: Número de documentos indexados.
* docs.deleted: Número de documentos marcados para ser borrados.
* store.size: Total de datos almacenados en el índice.
* pri.store.size: Datos almacenados en el sahrd primario.


## Creación de índices y mapping types

Vamos a crear un índice desde consola con curl para ver que información nos da Elasticsearch del índice que creemos.

In [ ]:
! curl -X PUT "http://elasticsearch:9200/twitter?pretty" -H 'Content-Type: application/json' -d' \
{ \
    "settings" : { \
        "index" : { \
            "number_of_shards" : 3, \
            "number_of_replicas" : 2 \
        } \
    } \
}'


Si queremos ver sólo los datos del índice creado podemos filtar el resultado de _cat/indices:

In [ ]:
!curl -X GET http://elasticsearch:9200/_cat/indices/twitter?v=true

Con la información anterior ¿sabrías contestar a estas preguntas?:

1. ¿Por qué la salud del cluster es yellow?
2. ¿Cuántos shards primarios tiene el índice?
3. ¿Cuantas réplicas hay de cada shard?
4. Busca esta misma información en la interfaz de Kibana.

Vamos a investigar un poco más por que la salud del cluster es yellow. Para ello vamos a consultar la información de los shards del índice que hemos creado.

In [ ]:
!curl -X GET http://elasticsearch:9200/_cat/shards/twitter?v=true

Las columnas que nos devuelve hacen referencia a la siguiente información:
* index: El nombre del índice.
* shard: El nombre del shard.
* prirep: 
    * p: primario
    * r: réplica
* state: Estado de la réplica:
    * INITIALIZING: El shard se está recuperando.
    * RELOCATING: El shard se está realojando en otro nodo del cluster.
    * STARTED: El shard está iniciado (funcionando normal).
    * UNASSIGNED: El shard no se ha asignado a ningún nodo.
* docs: Número de documentos almacenados en el shard.
* store: Tamaño del shard.
* ip: La ip del nodo donde está el shard.
* node: identificador del nodo donde está el shard.

Ahora vamos a ver el mapping type que tiene el índice que hemos creado.

In [ ]:
!curl -X GET http://elasticsearch:9200/twitter?pretty=true

Con la información obtenida:

1. ¿Cuántos mappings tiene el índice?
2. ¿Cuántos shards tiene el índice?
3. ¿Cuántas réplicas tiene el índice?

Vamos a insertar un dato en el índice.

In [ ]:
!curl -X POST "elasticsearch:9200/twitter/_doc/?pretty" -H 'Content-Type: application/json' -d' \
{ \
    "user" : "kimchy", \
    "post_date" : "2009-11-15T14:12:12", \
    "message" : "trying out Elasticsearch" \
}'

1. ¿Qué id le ha asignado Elasticsearch al documento?

Vamos a pedir a Elasticsearch otra vez los datos de índice.

In [ ]:
!curl -X GET http://elasticsearch:9200/twitter?pretty=true

Con la información obtenida:

1. ¿Cuantos mappings tiene ahora el índcie?
2. ¿De qué tipo es el campo message?
3. ¿De qué tipo es el campo date?
4. ¿De qué tipo es el campo user?

Si quieres profundizar en la API de Elasticsearch para poder extraer más información sobre la salud del cluster y el estado de sus índcies puedes consultar la documentación al respecto:
* https://www.elastic.co/guide/en/elasticsearch/reference/current/cat.html#cat
* https://www.elastic.co/guide/en/elasticsearch/reference/current/cluster.html#cluster

## Practica 1

Los siguientes ejercicios los vamos a realizar sobre Kibana, para ello entra en la siguiente URL: http://127.0.0.1:5601

También podrías realizarlos con el comando curl desde este notebook.

### 1. Crea los siguientes documentos desde la pestaña Dev Tools de kibana. 

Al insertar los documentos, les estamos asignando de forma explícita su identificador concatenando "/id" en la url de insercción de dato sen la api. Comprueba al insertar el documento que efectivamente le ha asignado el id indicado. Para ello revisa los datos que devuelve la API.

Puesto que no hemos creado el índice "mitienda" previemente Elasticsearch creará un mapping dinámico infiriendo el esquema del documento insertado.

`
PUT mitienda/_doc/1
{
"prenda" : "camiseta",
"marca" : "nike",
"color" : "rojo",
"talla" : 3,
"precio" : 50,
"descripcion" : "sin mangas",
"deporte" : ["baloncesto", "running"]
}
`

`
PUT mitienda/_doc/2
{
"prenda" : "camiseta",
"marca" : "adidas",
"color" : "rojo",
"talla" : 3,
"precio" : 25,
"descripcion" : "descatalogada",
"deporte" : ["futbol"]
}
`

`
PUT mitienda/_doc/3
{
"prenda" : "camiseta",
"marca" : "Nike",
"color" : "blanco",
"talla" : 4,
"precio" : 45,
"descripcion" : "manga larga y reversible",
"deporte" : ["futbol"]
}
`

`
PUT mitienda/_doc/4
{
"prenda" : "camiseta",
"marca" : "adidas",
"color" : "blanca",
"talla" : 4,
"precio" : 25,
"descripcion" : "manga corta",
"deporte" : ["running"]
}
`

### 2. Consulta el mapping type que ha creado Elasticsearch

* Utiliza la api de Elasticsearch como hemos visto previamente.
* O bien entra en la sección Menú > Management > Stack Management >  Data > Index Management y selecciona el índice "mitienda"


1. ¿Cuál es el type que le ha asignado a los campos del documento de tipo texto? 


### 3. Vamos a añaidr algunos documentos más, en este caso no vamos a indicar el id del documento de forma explícita

Igual que antes, puedes ejecutar estas consultas utilizando la herramienta de DevTools de Kibana o a través de la API de Elasticsearch utilizando el comando curl.

`
POST mitienda/_doc
{
"prenda" : "pantalón",
"marca" : "docker",
"color" : "negro",
"talla" : 36,
"precio" : 107,
"descripcion" : "pantalon skate",
"valoracion" : 3,
"deporte" : ["skate"]
}
`

`
POST mitienda/_doc
{
"prenda" : "camisa",
"marca" : "vans",
"color" : "azul",
"talla" : M,
"precio" : 70,
"descripcion" : "camisa urban",
"valoracion" : 4,
"deporte" : ["skate"]
}
`

1. ¿Por qué hemos tenido que cambiar el método de la petición a POST?
2. ¿Qué identificador les ha asignado a estos documentos Elasticsearch?

Vuelve a consultar el mapping de índice de "mitienda"
3. ¿Qué diferencias ha habido respecto al definición del anteriror mapping?


## Asignación de types dinámicamente

Como hemos visto antes, cuando no especificamos un mapping type explicito Elasticsearch lo genera dinámicamente haciendo inferencia de tipos de los campos de documento. 
De esta forma cuando Elsaticsearch encontraba un campo que contenía una cadena de texto le asignaba dinámicamente un type de tipo multi-field con el type text y el type keyword.

Vamos a modificar este comportamiento creando una dynamic template que le indique a Elasticsearch que cuando encuentre un campo del documento de tipo "String" lo mapee como un type keyword.

Para ello al crear el índice especificaremos la siguiente dynamic template (podemos añadir tantas como queramos):

`
"dynamic_templates": [
  {
    "strings_as_keywords": { 
      "match_mapping_type": "string", 
      "mapping": { "type": "keyword" } 
    } 
  }
]
`

## Practica 2:

Vamos a ponerlo en práctica:

### 1. Crea un índice mitienda-2 especificando el dynamic template que hemos visto antes:

In [ ]:
! curl -X PUT "http://elasticsearch:9200/mitienda-2?pretty" -H 'Content-Type: application/json' -d' \
{ \
    "settings" : { \
        "index" : { \
            "number_of_shards" : 3, \
            "number_of_replicas" : 1 \
        } \
    }, \
    "mappings": { \
      "dynamic_templates": [ \
        { \
          "strings_as_keywords": { \
            "match_mapping_type": "string", \
            "mapping": { "type": "keyword" } \
          } \
        } \
      ] \
    } \
}'


1. Consulta la información de índice para comprobar que se ha creado correctamente.

### 2. Vamos a insertar unos documentos en el índice creado

`
POST mitienda-2/_doc
{
"prenda" : "camiseta",
"marca" : "nike",
"color" : "rojo",
"talla" : 3,
"precio" : 50,
"descripcion" : "sin mangas",
"deporte" : ["baloncesto", "running"]
}
`

`
POST mitienda-2/_doc
{
"prenda" : "camiseta",
"marca" : "adidas",
"color" : "rojo",
"talla" : 3,
"precio" : 25,
"descripcion" : "descatalogada",
"deporte" : ["futbol"]
}
`

`
POST mitienda-2/_doc
{
"prenda" : "camiseta",
"marca" : "Nike",
"color" : "blanco",
"talla" : 4,
"precio" : 45,
"descripcion" : "manga larga y reversible",
"deporte" : ["futbol"]
}
`

`
POST mitienda/_doc
{
"prenda" : "camiseta",
"marca" : "adidas",
"color" : "blanca",
"talla" : 4,
"precio" : 25,
"descripcion" : "manga corta",
"deporte" : ["running"]
}
`

1. Vuelve a consultar el mapping del índice mitienda-2.
2. ¿Que type tienen ahora asociados los campos String del documento?

## Definir un mapping type específico para un índice

Ya hemos visto como trabajar con mappings dinámicos es hora de ver como definir y especificar un mapping para un índice.

Para ello a la hora de crear el índice debemos especificar el mapping type con el modelo de datos que hemos diseñado.

`
PUT my-explicit-index
{
  "mappings": {
    "properties": {
      "year": {
        "type": "integer"
      },
      "city": {
        "type": "keyword"
      },
      "country": {
        "type": "keyword"
      },
      "population_M":{
        "type": "float"
      },
      "attractions": {
        "type": "text"
      }
    }
  }
}
`

1. Consulta el mapping type creado.
2. Inserta 3 documentos que cumplan el mapping type definido. 

## Páctica 3

### Para terminar vamos a hacer el resto de operaciones CRUD sobre un índice de Elasticsearch.

Se llaman operaciones CRUD pro su acrónimo en inglés a las operaciones básicas sobre la únidad mínima de datos de un almacen de datos, en este caso sobre un documento:

* (C)reate -> Insertar un documento 
* (R)ead -> Leer un documento
* (U)pdate -> Modificar un documento
* (D)elete -> Borrar un documento

#### 1. Realizar una consulta de todos los documentos del índice mitienda

`
GET mitienda/_search
`


#### 2. Buscar el documento 1

`
GET mitienda/_doc/1
`

#### 3. Actualizar sólo el precio del documento 1 a 45

`
POST mitienda/_update/1
{
    "doc":{
        "precio" : 45
    }
}
`

* Ejecuta la sentencia que te permita averiguar si el documento se ha modificado correctamente.

`
GET mitienda/_doc/1
`

#### 4. Con la opción upsert modificaremos un documento si existe o lo creará si el documento no existe

Para que un update actue como un UPSERT, le pasaremos el parámetro "doc_as_upsert" a true:

`
POST mitienda/_update/20
{
    "doc":{
        "precio" : 90
    },
    "doc_as_upsert": true
}
`

* Ejecuta la sentencia que te permita comprobar que el documento se creado.
* ¿Cuántos camptos tiene este documento?
* Crea y ejecuta la sentencia necesaria para modificar un docuemto que ya exista usando el parámetro upsert.

`
GET mitienda/_doc/20
`

`
POST mitienda/_update/4
{
    "doc":{
        "precio" : 100
    },
    "doc_as_upsert": true
}
`

`
GET mitienda/_doc/4
`

#### 4. Modificar todos los documentos cuyo color sea rojo y cambiar su color a morado

Para modificar el conjunto de documentos que cumplan una condición utilizaremos la operación "update_by_query" indicando en el body de la petición la consulta con la condición que tienen que cumplir los documentos para ser modificados. 

Acemás de la consulta tendremos que pasarle el script que genera la modificación. En este caso usaremos el lenguaje "painless" de elasticsearch que utilizaremos a menudo en los notebooks:
* **source** representa el escript que queremos ejecutar para realizar la modificación. En este caso queremos modificar el campo color del documento original (ctx._source). ctx representa el contexto de la query, es decir cada uno de los match devueltos por la query y su campo _source hace referencia al documento original del match. 
* **lang** indica el lenguaje de scripting que estamos usando.
* **params**  listado de parámetros que se le pasa al script del campo source.

`
POST mitienda/_update_by_query
{
  "query": {
    "term": {
      "color.keyword": "rojo"
    }
  },
  "script" : {
    "source": "ctx._source.color = params.newValue",
    "lang": "painless",
    "params": {
      "newValue": "morado"
    }
  }
}
`

* Ejecuta la sentencia que te permita comprbar que se han modificado correctamente los documentos que cumplen la condición.

`
GET mitienda/_search
`

#### 5. Modificar los documentos cuya marca sea adidas para añadirle el campo descuento con valor 10

Utilizaremos, al igual que en el ejercicio anterior, la operación "update_by_query" indicándole el script que realiza la operación de modificación que queremos realizar sobre los documentos que cumplan la condición de la query.

`
POST mitienda/_update_by_query
{
  "query": {
    "term": {
      "marca.keyword": "adidas"
    }
  },
  "script" : {
    "source": "ctx._source.descuento = params.newValue",
    "lang": "painless",
    "params": {
      "newValue": 10
    }
  }
}
`

* Ejecuta la sentencia que te permita comprbar que se han modificado correctamente los documentos que cumplen la condición.

`
GET mitienda/_search
`

#### 6. Realizar una query que devuelva todos los documentos del índice

Elasticserach por defecto sólo devuelve 10 docuementos. En los siguientes Notebooks veremos como funciona esta opearción en detalle.

`
GET mitienda/_search
{
  "query": {
    "match_all": {}
  }
}
`

* ¿Cuántos elementos devuelve?
* Basándote en la consutla del apartado anterior intenta realizar una consulta que devuleva todos los documentos cuyo color sea morado.

`
GET mitienda/_search
{
  "query": {
    "term": {
      "color.keyword": "morado"
    }
  }
}
`

#### 7. Borra el documento 1

`
DELETE mitienda/_doc/1
`

* Ejecuta la sentencia adecuada para comprobar que el documento se ha borrado correctamente.

`
GET mitienda/_doc/1
`

#### 8. Borrar todos los documentos cuya marca sea adidas

`
POST /mitienda/_delete_by_query
{
  "query": {
    "match": {
      "marca.keyword": "adidas"
    }
  }
}
`

* Realiza la consulta necesaria para comprobar que se han borrado los documentos adecuadamente. 

`
GET mitienda/_search
`

o

`
GET mitienda/_search
{
  "query": {
    "term": {
      "marca.keyword": "adidas"
    }
  }
}
`